In [ ]:
from pathlib import Path
import time

import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt

from src.models.fetch_data_2d import get_tf_dataset, tf_parse_image, parse_image

In [ ]:
path_data_nii = Path("/home/val/python_wkspce/plc_seg/data/interim/nii_raw")
path_mask_lung_nii = Path(
    "/home/val/python_wkspce/plc_seg/data/interim/lung_contours")

patient_list = [
    f.name.split("__")[0] for f in path_mask_lung_nii.rglob("*LUNG*")
]

patient_ds = tf.data.Dataset.from_tensor_slices(patient_list)

n = 2
sradius = 3
t1 = time.time()
for p in patient_ds.take(n):
    results = parse_image(
        p,
        path_data_nii,
        path_mask_lung_nii,
        spacing=(1, 1, 1),
        ct_window_str="lung",
        output_shape=(256, 256),
        random_center=True,
        augment_mirror=True,
        augment_angles=(20, 20, 45),
        interp_order=3,
        mask_smoothing=True,
        smoothing_radius=sradius,
    )

In [ ]:
results[0].shape

In [ ]:
plt.imshow(results[0][:,:,0],cmap='gray')
plt.imshow(results[1][:,:,0], cmap='jet', alpha=0.5)

In [ ]:
len(patient_list)

In [ ]:
patient_list = patient_list[:1]

In [ ]:
data =  get_tf_dataset(
    patient_list,
    path_data_nii,
    path_mask_lung_nii,
    random_center=True,
    augment_angles=(20,20,45),
    augment_mirror=False,
    num_parallel_calls=None,
    output_shape=(256,256),
    spacing=(1, 1, 1),
    ct_window_str="lung",
    return_patient_name=True,
    interp_order=3,
    mask_smoothing=True,
    smoothing_radius=3,
    )

In [ ]:
X = data.batch(1).take(1).as_numpy_iterator()

In [ ]:
image, mask, patient = next(X)

In [ ]:
image.shape

In [ ]:
b=0
msk = mask[b,:,:, 0]

In [ ]:
print(f"patient: {patient[b]}")
c=0 # 0:ct, 1:pt
c_m=0 # 0: gtv t, 1:gtv l, 2: lung1, 3: lung2


plt.imshow(image[b,:,:, c],cmap='gray')
plt.imshow(mask[b,:,:,c_m], cmap='jet', alpha=0.5)

In [ ]:
for i in range(1):
    start = time.time()
    image, mask = next(X)
    print(f"image number {i}")
    print(image.shape)
    end = time.time()
    print(f"time to processe it {end - start}")


In [ ]:
plt.imshow(image[0,:,:,72, 0])